# Clean Data

In [49]:
from pyspark.sql.functions import to_timestamp,year, month

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 51, Finished, Available)

## Read file from bronze layer

In [1]:
df = spark.read.format("csv").option("header","true").load("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/bf01a96a-f669-4bd4-a676-8dbd2382d089/Files/full_load.csv")



StatementMeta(, 05e5e692-b0a8-4570-b99b-104544c7b31c, 3, Finished, Available)

## Change order date and shipping date to datetime type

In [ ]:
    
df = df.withColumn("order date (DateOrders)", to_timestamp("order date (DateOrders)", "M/d/yyyy H:mm"))
df = df.withColumn("shipping date (DateOrders)", to_timestamp("shipping date (DateOrders)", "M/d/yyyy H:mm"))

## Create a copy 

In [51]:
# Create a copy of values
dataco = df 

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 53, Finished, Available)

## Create a function to seperate columns

In [52]:
#Tạo một function để tách bảng


def seperate_table(df, lst_table):
    
    new_df = df.select(*lst_table)
    return new_df

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 54, Finished, Available)

# Create Orders detail table

In [53]:
#import mấy thư viện cần thiết
from pyspark.sql.functions import concat,col,lit,cast, sum


StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 55, Finished, Available)

## Create a function to clean data

In [54]:
import re

def sanitize_column_name(column_name):
    # Replace spaces with underscores
    sanitized_name = column_name.replace(' ', '_')
    # Convert to lowercase
    sanitized_name = sanitized_name.lower()
    return sanitized_name

# Get the current column names
columns = dataco.columns

# Create a dictionary to map old column names to sanitized names
renaming_dict = {col: sanitize_column_name(col) for col in columns}

# Rename columns in the DataFrame
for old_name, new_name in renaming_dict.items():
    dataco = dataco.withColumnRenamed(old_name, new_name)


# renamed thằng type thành type of transaction cho dễ hiểu hơn
dataco = dataco.withColumnRenamed('Type','type_of_transaction')

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 56, Finished, Available)

## change data type of columns

In [56]:
def change_type(data, columns, types):
    for col in columns:
        data  = data.withColumn(col, data[col].cast(types))

    return data

columns_to_change_int = ["days_for_shipping_(real)","days_for_shipment_(scheduled)","Late_delivery_risk","order_item_quantity"]
dataco = change_type(dataco,columns_to_change_int, "int")  


columns_to_change_floats = ["benefit_per_order", "sales_per_customer", "latitude", "longitude",
 "order_item_discount", "order_item_discount_rate", "order_item_product_price",
 "order_item_profit_ratio", "sales", "order_profit_per_order", "product_price"]


dataco = change_type(dataco, columns_to_change_floats,"float")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 58, Finished, Available)

In [57]:
# lấy những thông tin cần cho bảng orders

columns_order_table = ['order_id', 'customer_id','product_card_id' ,'order_date_(dateorders)', 
'shipping_date_(dateorders)', 'shipping_mode', 'delivery_status', 
'type_of_transaction', 'late_delivery_risk', 'order_status', 'market', 
'order_region', 'order_country', 'order_state', 'order_city', 'customer_country', 
'customer_city', 'customer_street', 'sales', 'benefit_per_order', 'order_item_discount',
'order_item_quantity', 'order_profit_per_order', "latitude", "longitude"]
order_details = seperate_table(dataco, columns_order_table)

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 59, Finished, Available)

In [58]:
from pyspark.sql.functions import sum as spark_sum
import pyspark.sql.functions as F

from pyspark.sql.functions import concat_ws,when

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 60, Finished, Available)

In [59]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import filter

from pyspark.sql.functions import row_number

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 61, Finished, Available)

In [60]:
order_details = order_details.withColumn("concat_territory", concat_ws("_", col("market"), col("order_region"), col("order_country"),col("order_state"), col("order_city")))


# làm tương tự như thằng territory
order_details = order_details.withColumn("concat_store_address", concat_ws("_", col("customer_country"), col("customer_city"),col("customer_street")))

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 62, Finished, Available)

# Create territory table

In [61]:
#tạo bảng territory
territory = order_details.select("concat_territory","market", "order_region","order_country", "order_state", "order_city")
# drop những giá trị trùng lặp từ bảng

territory = territory.dropDuplicates(subset=["concat_territory","market", "order_region","order_country", "order_state", "order_city"])

# rename để tránh trường hợp  ambitigous khi join 
territory  = territory.withColumnRenamed("concat_territory" , "ter_concat_territory")
# kiểm tra xem có bao nhiêu dòng trong territory
territory.count()

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 63, Finished, Available)

3772

# Create store table 

In [62]:


columns_store_table = ["concat_store_address", "customer_country" , "customer_city" , "customer_street", "latitude", "longitude" ]

store_table = order_details.select(columns_store_table)

store_table = store_table.dropDuplicates(subset=["concat_store_address", "customer_country" , "customer_city" , "customer_street"])

store_table = store_table.withColumnRenamed("concat_store_address" , "st_concat_store_address")


StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 64, Finished, Available)

In [63]:
duplicates_store = store_table.groupBy("st_concat_store_address").count().filter(col("count") > 1)
duplicates_store.show()

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 65, Finished, Available)

+-----------------------+-----+
|st_concat_store_address|count|
+-----------------------+-----+
+-----------------------+-----+



In [64]:
store_column_renamed = {
    "customer_country" : "store_country" ,
    "customer_city" : "store_city" , 
    "customer_street" : "store_street"
}

for old_name, new_name in store_column_renamed.items():
    store_table = store_table.withColumnRenamed(old_name,new_name)

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 66, Finished, Available)

In [65]:
store_table.show(5)

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 67, Finished, Available)

+-----------------------+-------------+-----------+--------------------+---------+----------+
|st_concat_store_address|store_country| store_city|        store_street| latitude| longitude|
+-----------------------+-------------+-----------+--------------------+---------+----------+
|   EE. UU._Loveland_...|      EE. UU.|   Loveland|   874 Thunder Front| 39.27056| -84.27059|
|   Puerto Rico_Cagua...|  Puerto Rico|     Caguas|2585 Silent Autum...| 18.21335|-66.370575|
|   Puerto Rico_Cagua...|  Puerto Rico|     Caguas|8263 Indian Crossing|18.292559| -66.37062|
|   EE. UU._Allentown...|      EE. UU.|  Allentown|     6779 Iron Woods|40.603493|-75.472435|
|   EE. UU._Los Angel...|      EE. UU.|Los Angeles|3422 Jagged Willo...| 34.09961|-118.35074|
+-----------------------+-------------+-----------+--------------------+---------+----------+
only showing top 5 rows



In [66]:
order_details = order_details.drop("market","order_region","order_country","order_state",  "order_city", "customer_country", "customer_city", "customer_street", "latitude", "longitude")


StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 68, Finished, Available)

# Create product table

In [67]:
lst_product = ["product_card_id" , "product_name", "product_price", "product_status", "category_id"]


product = seperate_table(dataco,lst_product)

product = product.dropDuplicates(subset = lst_product)


display(product)


StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 69, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2634451b-1510-4e35-be4b-e7dd0a56d1d8)

# Create Category table

In [68]:
lst_category = ["category_id", "category_name", "department_id"]

category  = seperate_table(dataco, lst_category)
category = category.dropDuplicates(subset= lst_category)

category.show()

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 70, Finished, Available)

+-----------+--------------------+-------------+
|category_id|       category_name|department_id|
+-----------+--------------------+-------------+
|          2|              Soccer|            2|
|         45|             Fishing|            7|
|         59|              Books |            8|
|         63| Children's Clothing|            4|
|         71|               Music|            9|
|         11| Fitness Accessories|            3|
|         38|    Kids' Golf Clubs|            6|
|         76|    Women's Clothing|            4|
|          3| Baseball & Softball|            2|
|         36|          Golf Balls|            6|
|         10|   Strength Training|            3|
|         37|         Electronics|            6|
|         61|                CDs |            9|
|         65|Consumer Electronics|           10|
|         24|     Women's Apparel|            5|
|         66|              Crafts|            4|
|         73|      Sporting Goods|            2|
|         41|       

# Create Department table

In [69]:
lst_department = ["department_id", "department_name"]

department  = seperate_table(dataco,lst_department)


department = department.dropDuplicates(subset=lst_department)
department.show()

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 71, Finished, Available)

+-------------+------------------+
|department_id|   department_name|
+-------------+------------------+
|           11|          Pet Shop|
|            6|          Outdoors|
|            8|         Book Shop|
|            3|          Footwear|
|           12|Health and Beauty |
|            2|           Fitness|
|            7|          Fan Shop|
|           10|        Technology|
|            4|           Apparel|
|            5|              Golf|
|            9|        Discs Shop|
+-------------+------------------+



# Create Transaction Table

In [70]:

transaction = seperate_table(dataco, ["type_of_transaction"])
transaction = transaction.dropDuplicates(subset = ["type_of_transaction"])
transaction.show()

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 72, Finished, Available)

+-------------------+
|type_of_transaction|
+-------------------+
|           TRANSFER|
|               CASH|
|            PAYMENT|
|              DEBIT|
+-------------------+



# Create Shipping Mode table

In [71]:
shipping_mode = seperate_table(dataco, ["shipping_mode"])
shipping_mode = shipping_mode.dropDuplicates(subset = ["shipping_mode"])
shipping_mode.show()

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 73, Finished, Available)

+--------------+
| shipping_mode|
+--------------+
|   First Class|
|      Same Day|
|  Second Class|
|Standard Class|
+--------------+



# Create order status

In [72]:
order_status = seperate_table(dataco , ["order_status"])
order_status = order_status.dropDuplicates(["order_status"])

order_status.show()

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 74, Finished, Available)

+---------------+
|   order_status|
+---------------+
|PENDING_PAYMENT|
|       COMPLETE|
|        ON_HOLD|
| PAYMENT_REVIEW|
|     PROCESSING|
|         CLOSED|
|SUSPECTED_FRAUD|
|        PENDING|
|       CANCELED|
+---------------+



# Create delivery status

In [73]:
delivery_status = seperate_table(dataco , ["delivery_status"])
delivery_status = delivery_status.dropDuplicates(["delivery_status"])

delivery_status.show()

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 75, Finished, Available)

+-----------------+
|  delivery_status|
+-----------------+
| Shipping on time|
| Advance shipping|
|Shipping canceled|
|    Late delivery|
+-----------------+



# Create late_delivery_risk table 

In [74]:
delivery_risk = seperate_table(dataco, ["late_delivery_risk"])
delivery_risk = delivery_risk.dropDuplicates(subset = ["late_delivery_risk"])

delivery_risk.show()

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 76, Finished, Available)

+------------------+
|late_delivery_risk|
+------------------+
|                 1|
|                 0|
+------------------+



# Create Customer Table 

In [75]:
lst_customers = ["customer_id", "customer_fname", "customer_lname", "customer_segment"]


customers = seperate_table(dataco, lst_customers)
customers = customers.dropDuplicates(subset = lst_customers)
customers = customers.withColumn("customer_fullname", concat(col("customer_fname"), lit(" "), col("customer_lname")))
customers = customers.drop("customer_fname", "customer_lname")
customers.show()


StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 77, Finished, Available)

+-----------+----------------+-----------------+
|customer_id|customer_segment|customer_fullname|
+-----------+----------------+-----------------+
|       5113|        Consumer|   Judith Osborne|
|       8850|       Corporate|    Crystal Smith|
|       9876|     Home Office|       Mary Smith|
|      12416|        Consumer|      Aaron Cline|
|      10116|        Consumer|     Ashley Smith|
|       7216|        Consumer|     Mary Hendrix|
|       4524|        Consumer|    Johnny Cooper|
|       1662|       Corporate|     George Smith|
|       8763|     Home Office|  Pamela Williams|
|      17911|        Consumer|    Alyssa Carter|
|       3513|       Corporate|    Shirley Boyer|
|       3817|       Corporate|      Gary Garcia|
|       9240|        Consumer|     Janice Brown|
|       4181|        Consumer|     Aaron Hudson|
|      10664|       Corporate|        Mary Bush|
|       4645|       Corporate|     Steven Smith|
|      10220|        Consumer|     Gerald Mason|
|       7954|       

# Load to silver

In [76]:
order_renamed = {
    "days_for_shipping_(real)" : "days_for_shipping_real",
    "days_for_shipment_(scheduled)": "days_for_shipment_scheduled",
    "shipping_date_(dateorders)" : "shipping_date_dateorders" ,
    "order_date_(dateorders)" : "order_date_dateorders"
     

}
for old_name , new_name in order_renamed.items():
    order_details = order_details.withColumnRenamed(old_name,new_name)

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 78, Finished, Available)

## Load order details

In [77]:
# tao partition de phan du lieu theo year month
order_details = order_details.withColumn("year", year('order_date_dateorders'))
order_details = order_details.withColumn("month", month('order_date_dateorders'))


order_details.write \
    .mode("append") \
    .partitionBy("year", "month") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/orders")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 79, Finished, Available)

## Load Product

In [78]:
product.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/product")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 80, Finished, Available)

## Load Category

In [79]:
category.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/category")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 81, Finished, Available)

## Load Department

In [80]:
department.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/department")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 82, Finished, Available)

## Load territory

In [81]:
territory.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/territory")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 83, Finished, Available)

## Load store region 

In [82]:
store_table.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/store")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 84, Finished, Available)

## Load Customers

In [83]:
customers.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/customer")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 85, Finished, Available)

## load transaction

In [84]:
transaction.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/transaction")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 86, Finished, Available)

## Load shipping mode

In [85]:
shipping_mode.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/shipping_mode")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 87, Finished, Available)

## Load order status

In [86]:
order_status.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/order_status")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 88, Finished, Available)

## Load delivery status

In [87]:
delivery_status.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/delivery_status")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 89, Finished, Available)

## Load delivery_risk


In [88]:
delivery_risk.write \
    .mode("append") \
    .format("csv") \
    .option("header", "true") \
    .save("Files/delivery_risk")

StatementMeta(, a99e1057-6666-4e31-b7a9-2d3817fb2d9e, 90, Finished, Available)